In [1]:
import numpy as np
from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, BatchNormalization, Reshape, Conv1D, Conv2D, \
MaxPooling2D, Concatenate, Dropout, Input, Flatten
import tensorflow as tf
import pandas as pd
import subprocess
import os
os.chdir('../input/shared-files/utils')
from fileHandling import getData
from fileHandling import splitData
from BLAS import create_dataset
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler
path_parent = os.path.dirname(os.getcwd())
os.chdir(path_parent)
os.chdir(path_parent)
os.chdir('Data')

In [2]:
def process_data(name):
    SeriesName     = name
    Strategy       = 'Differenced' # Differenced, Returns
    Lag = 5
    Dates      = ['2020-02-28',     '2020-06-01']
    Series, Transformed_Series = getData(SeriesName + '.csv', Strategy)

    Training,      Validation,      Testing      = splitData(Series,             Dates)
    Training_diff, Validation_diff, Testing_diff = splitData(Transformed_Series, Dates)
    # -Traditional Series
    Validation      = pd.concat([Training.iloc[-Lag:],   Validation])
    Testing         = pd.concat([Validation.iloc[-Lag:], Testing])
    # -Differenced Series
    Validation_diff = pd.concat([Training_diff.iloc[-Lag:],   Validation_diff])
    Testing_diff    = pd.concat([Validation_diff.iloc[-Lag:], Testing_diff])

    trainX, trainY = create_dataset(Training_diff,   Lag, SeriesName)
    validX, validY = create_dataset(Validation_diff, Lag, SeriesName)
    testX,  testY  = create_dataset(Testing_diff,    Lag, SeriesName)

    scaler = StandardScaler()
    trainX = np.expand_dims( scaler.fit_transform(trainX[:,:,0]), axis=-1)
    validX = np.expand_dims( scaler.fit_transform(validX[:,:,0]), axis=-1)
    testX  = np.expand_dims( scaler.fit_transform(testX[:,:,0]),  axis=-1)
    
    return trainX,trainY, validX,validY, testX,testY

In [3]:
trainX_eth,trainY_eth, validX_eth,validY_eth, testX_eth,testY_eth = process_data('ETH')
trainX_btc,trainY_btc, validX_btc,validY_btc, testX_btc,testY_btc = process_data('BTC')
trainX_xrp,trainY_xrp, validX_xrp,validY_xrp, testX_xrp,testY_xrp = process_data('XRP')

In [4]:
def lstmcomp(modinput, in_shape, k1 = 1, k2 = 2, k3 = 3, conv1dmaps = 16, 
             lay_padding = 'SAME', layer_act = 'relu'):
    conv1 = Conv1D(conv1dmaps,k1,padding = lay_padding, activation=layer_act)(modinput)
    conv2 = Conv1D(conv1dmaps,k2,padding = lay_padding, activation=layer_act)(modinput)
    conv3 = Conv1D(conv1dmaps,k3,padding = lay_padding, activation=layer_act)(modinput)
    dep = Concatenate()([conv1, conv2, conv3, modinput]) # Might just flatten this layer and feed it to
    # the lstm
    re1 = Reshape((in_shape, (conv1dmaps * 3) + 1, 1))(dep)
    conv2d = Conv2D(1, (1,1), padding = lay_padding, activation=layer_act)(re1)
    flat = Flatten()(conv2d)
    re2 = Reshape((flat.shape[1],1))(flat)
    # Did not add a dense layer here (might add it, but IDK if it is necessary)
    lstm = LSTM(50)(re2)
    #flat = Flatten()(conv2d)
    #lstm = LSTM(50)(conv2d)
    return lstm

inshape = 5
input1 = Input(shape=(inshape, 1))
input2 = Input(shape=(inshape, 1))
input3 = Input(shape=(inshape, 1))
out1 = lstmcomp(input1, inshape)
out2 = lstmcomp(input2, inshape)
out3 = lstmcomp(input3, inshape)

concat = Concatenate()([out1,out2,out3])
dense1 = Dense(256, activation = 'relu')(concat)
batchnorm1 = BatchNormalization()(dense1)
drop1 = Dropout(3/10)(batchnorm1)
dense2 = Dense(64)(drop1)
batchnorm2 = BatchNormalization()(dense2)
drop2 = Dropout(2/10)(batchnorm2)

model = Model(inputs = [input1, input2, input3], outputs = drop2)
#model.summary()

In [5]:
#%%
# Checkpoint
checkpoint = ModelCheckpoint("model/model.hdf5", 
                              monitor        = 'val_loss', 
                              verbose        = 0, 
                              save_best_only = True, 
                              mode           = 'min')

# Earlystopping
earlystopping = EarlyStopping(monitor       = 'val_loss', 
                              mode          = 'min', 
                              verbose       = 1, 
                              patience      = 10)

# Learning rate adjustment
# lrs_scheduler = step_decay_schedule(initial_lr=1e-4, decay_factor=0.75, step_size=2)
lrs_scheduler  = ReduceLROnPlateau(monitor     = 'val_loss', 
                                   factor      = 0.75,
                                   patience    = 5)

In [6]:
# Need to finish this part still
path_parent = os.path.dirname(os.getcwd())
os.chdir(path_parent)
path_parent = os.path.dirname(os.getcwd())
os.chdir(path_parent)
path_parent = os.path.dirname(os.getcwd())
os.chdir(path_parent)
path_parent = os.path.dirname(os.getcwd())
os.chdir(path_parent)

epochs         =  20
batch_size     =  8
model.compile(loss='mean_squared_error', optimizer='adam')
score = model.fit(x = [trainX_eth, trainX_btc, trainX_xrp], 
                  y = trainY_btc,
                  epochs          = epochs, 
                  batch_size      = batch_size, 
                  callbacks       = [checkpoint, earlystopping, lrs_scheduler],
                  verbose         = 1, 
                  validation_data = ([validX_eth, validX_btc, validX_xrp], validY_btc))

In [ ]:
'''
model.load_weights('model/model.hdf5')

y_pred = model.predict(testX)[:,0]



# Set DataFrame with 'Real' and 'Predicted' values
Prices = pd.DataFrame([], columns=[SeriesName, 'Predict'])
# Get real values

# Reverse transformation
if (Strategy == 'Differenced'):
    Prices[ SeriesName ]    = Testing[SeriesName][Lag:]
    Prices['Predict'] = Testing[SeriesName][Lag-1:-1].to_numpy() + y_pred
else:
    Prices[ SeriesName ] = np.exp( Testing[SeriesName][Lag:] )
    Prices['Predict']    = np.exp( Testing[SeriesName][Lag-1:-1].to_numpy() + y_pred )
'''

In [ ]:

'''
# Regression performance
#
MAE, RMSE, MAPE, SMAPE, R2 = RegressionEvaluation( Prices )
#
print('MAE   = %.3f' % MAE)
print('RMSE  = %.3f' % RMSE)
print('MAPE  = %.3f' % MAPE)
print('SMAPE = %.3f' % SMAPE)
print('R2    = %.3f' % R2)
print('\n')



# Classification performance
#
CM, Accuracy, AUC, F1, GM, Sen, Spe, PPV, NPV = ClassificationEvaluation( Prices )
#
print('Accuracy  = %.2f%%' % (100*Accuracy))
print('AUC       = %.3f' % AUC)
print('F1        = %.3f' % F1)
print('GM        = %.3f' % GM)
print('Sen       = %.3f' % Sen)
print('Spe       = %.3f' % Spe)
print('Spe x Sen = %.3f' % (Sen*Spe))
print('\n')


# Confusion matrix
#
ConfusionMatrixVisualize(CM)
'''
